In [8]:
%reset -f

In [9]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [12]:
import ee

ee.Initialize(project='ee-andrewfullhart')

#Units are in mm for all datasets
#ee.ImageCollection('NASA/ORNL/DAYMET_V4') //prcp
#ee.ImageCollection('OREGONSTATE/PRISM/AN81d') //ppt
#ee.ImageCollection('IDAHO_EPSCOR/GRIDMET') //pr
#ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA') //pr

product = 'maca'

dataset = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')                     \
            .filter(ee.Filter.date('1974-01-01', '2014-01-01'))

ic = dataset.select('pr')

rn_point = ee.Geometry.Point(-119.7683, 39.5078)
lv_point = ee.Geometry.Point(-115.1633, 36.0719)
sl_point = ee.Geometry.Point(-111.9650, 40.7706)
mb_point = ee.Geometry.Point(-109.5458, 38.5744)
tb_point = ee.Geometry.Point(-110.0686, 31.7119)
ab_point = ee.Geometry.Point(-106.6156, 35.0419)
fs_point = ee.Geometry.Point(-111.6664, 35.1442)
jd_point = ee.Geometry.Point(-106.7403, 32.6161)

points = ee.List([rn_point, lv_point, sl_point, mb_point, tb_point, ab_point, fs_point, jd_point])
pt_labels = ['rn', 'lv', 'sl', 'mb', 'tb', 'ab', 'fs', 'jd']


for i in range(8):

  point = ee.Geometry(points.get(i))
  station = pt_labels[i]
  prop_list = ic.getRegion(point, 500)

  def props_fn(props):
    props = ee.List(props)
    return ee.Feature(None, {'date':props.get(0), 'precip':props.get(4)})

  tableFeat = ee.FeatureCollection(prop_list.slice(1).map(props_fn))

  task = ee.batch.Export.table.toDrive(
          collection = tableFeat,
          description='{}_{}'.format(station, product),
          folder='GEE_Downloads',
          selectors=['date', 'precip'])

  task.start()


